## VQGAN

### Download checkpoints

- Download pre-trained VQGAN checkpoint from https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/
- Download pre-trained LPIPS checkpoint from https://heibox.uni-heidelberg.de/f/607503859c864bc1b30b/?dl=1

### Load model

In [ ]:
import sys
sys.path.append('../src')

In [ ]:
import torch

from torchvision import transforms
from torch.utils.data import DataLoader

from vqgan.model import VQModel
from data import VQVisualNewsDataset

In [ ]:
ddconfig = {
    'double_z': False,
    'z_channels': 256,
    'resolution': 256,
    'in_channels': 3,
    'out_ch': 3,
    'ch': 128,
    'ch_mult': [1, 1, 2, 2, 4],
    'num_res_blocks': 2,
    'attn_resolutions': [16],
    'dropout': 0.0
}

lossconfig = {
    'disc_conditional': False,
    'disc_in_channels': 3,
    'disc_start': 0,
    'disc_weight': 0.75,
    'disc_num_layers': 2,
    'codebook_weight': 1.0,
    'lpips_ckpt_path': '../src/checkpoints/lpips/vgg.pth'
}

config = {'ddconfig': ddconfig, 'lossconfig': lossconfig, 'n_embed': 16384, 'embed_dim': 256}

In [ ]:
# ckpt = torch.load('../src/checkpoints/vqgan/last.ckpt', map_location='cpu')
# ckpt.keys()
# # dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'callbacks', 'optimizer_states', 'lr_schedulers', 'state_dict'])

In [ ]:
model = VQModel.from_checkpoint('../src/checkpoints/vqgan/last.ckpt', config)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
train_set = VQVisualNewsDataset('../src/data/visual_news_mini', transform)

In [ ]:
train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=0, pin_memory=True)

In [ ]:
import io

import torchvision
import numpy as np
import matplotlib.pyplot as plt
import requests

from PIL import Image


def download_image(url):
    resp = requests.get(url)
    resp.raise_for_status()
    return Image.open(io.BytesIO(resp.content))


def imshow(img, title=None):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    img = np.transpose(npimg, (1, 2, 0))
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.axis('off')

    
def show_random_batch(data_loader):
    # Get a random batch
    images = next(iter(data_loader))

    imshow(torchvision.utils.make_grid(images))
    plt.show()


def visualize_model_batch(batch):
    # Encoded image tokens
    quant_states, loss, info = model.encode(batch)
    
    # Decode image tokens, i.e. reconstruct image from image tokens
    rec = model.decode(quant_states)
    
    # Display
    imshow(torchvision.utils.make_grid(batch), 'Original')
    plt.show()
    imshow(torchvision.utils.make_grid(rec.detach()), 'Reconstructed')
    plt.show()


def visualize_model(data_loader):
    # Get a random batch
    images = next(iter(data_loader))
    
    visualize_model_batch(images)

In [ ]:
show_random_batch(train_loader)

In [ ]:
visualize_model(train_loader)

In [ ]:
# Download an image, encode it, and then reconstruct

# Load the image
sample_img = download_image('https://heibox.uni-heidelberg.de/f/7bb608381aae4539ba7a/?dl=1')

# Preprocess the image using the transformation pipeline
sample_tensor = transform(sample_img)

# Add a batch dimension to the tensor
sample_batch = sample_tensor.unsqueeze(0)

visualize_model_batch(sample_batch)

## Train

In [ ]:
# Set up the optimizer for the autoencoder and discriminator
base_lr = 4.5e-06
opt_ae, opt_disc = model.configure_optimizers(learning_rate=base_lr)

opt_states = torch.load('../src/checkpoints/vqgan/last.ckpt', map_location='cpu')['optimizer_states']
opt_ae.load_state_dict(opt_states[0])
opt_disc.load_state_dict(opt_states[1])

In [ ]:
# Training loop

for epoch in range(1):
    print('\nEpoch:', epoch + 1)
    for i, inputs in enumerate(train_loader):
        # Update the autoencoder
        aeloss, log_dict_ae = model.training_step(inputs, 0)
        
        # Update the discriminator
        discloss, log_dict_disc = model.training_step(inputs, 1)
        
        # Update global step
        model.step()
        
        # Print statistics every 2 mini-batches
        if i % 2 == 0:
            print('  step:', i, 'ae_loss:', aeloss.item(), 'disc_loss:', discloss.item())
        
        # For now, let's train for 10 mini-batches
        if i >= 10:
            break

In [ ]:
visualize_model(train_loader)

In [ ]:
visualize_model_batch(sample_batch)

In [ ]:
#save model
PATH = "../src/checkpoints/vqgan/finetuned.ckpt"

torch.save({
    'epoch': 1,
    'optimizer_states': [opt_ae.state_dict(), opt_disc.state_dict()],
    'lr_schedulers': base_lr,
    'state_dict': model.state_dict()
}, PATH)

In [ ]:
#load model
checkpoint = torch.load(PATH, map_location='cpu')
model.load_state_dict(checkpoint['state_dict'])
opt_states = checkpoint['optimizer_states']
opt_ae.load_state_dict(opt_states[0])
opt_disc.load_state_dict(opt_states[1])
epoch = checkpoint['epoch']

In [ ]:
visualize_model(train_loader)